In [1]:
import pandapower.networks as pn

net = pn.create_cigre_network_lv()

In [2]:
net.switch.loc[1, "closed"] = False
net.switch.loc[2, "closed"] = False

In [3]:
# Iterate over all loads in the network and set controllable to False
for load_idx in net.load.index:
    net.load.at[load_idx, 'controllable'] = False

In [4]:
# Set voltage limits for all buses
for bus_idx in net.bus.index:
    net.bus.at[bus_idx, 'min_vm_pu'] = 0.95  # Set the minimum voltage limit to 0.95 pu
    net.bus.at[bus_idx, 'max_vm_pu'] = 1.05  # Set the maximum voltage limit to 1.05 pu

In [5]:
# Set line loading limits for all lines
for line_idx in net.line.index:
    net.line.at[line_idx, 'max_loading_percent'] = 100  # Set the maximum loading limit to 100%

In [6]:
import numpy as np
import pandas as pd

import pandapower as pp
import pandapower.control as control
import pandapower.networks as nw
import pandapower.timeseries as timeseries
from pandapower.timeseries import OutputWriter, DFData
from pandapower.control import ConstControl

In [7]:
# Load and preprocess the PV generation profile CSV file
df_pv = pd.read_csv("pv_generation_profile.csv")
df_pv['time'] = pd.to_datetime(df_pv['time'], format='%H:%M:%S').dt.time
df_pv['time_step'] = range(len(df_pv))  # Create a numerical index
df_pv.set_index('time_step', inplace=True)
df_pv['pvgen'] = df_pv['pvgen'] * 250 / 1000
ds_pv = DFData(df_pv)

In [8]:
# Load and preprocess the load profile CSV file
df = pd.read_csv("load_profile_1111.csv")
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time
df['time_step'] = range(len(df))  # Create a numerical index
df.set_index('time_step', inplace=True)
df['mult'] = df['mult'] * 15 / 1000
ds = DFData(df)

In [9]:
profile_loads = net.load.index.intersection([0, 1, 2, 3, 4, 5])
const_load = ConstControl(net, element='load', element_index=profile_loads,
                          variable='p_mw', data_source=ds, profile_name=["mult"] * len(profile_loads))

In [10]:
# Add PV generators to the corresponding buses
pv_buses = [12, 16, 17, 18, 19]
pv_generators = []
for bus in pv_buses:
    pv_gen = pp.create_sgen(net, bus, p_mw=0, q_mvar=0, type='pv', controllable=True)
    # Set initial limits
    net.sgen.at[pv_gen, 'min_p_mw'] = 0
    net.sgen.at[pv_gen, 'max_p_mw'] = df_pv['pvgen'].max()  # Set this to the maximum possible value in the profile
    net.sgen.at[pv_gen, 'min_q_mvar'] = -0.5/1000  # Example value, adjust as needed
    net.sgen.at[pv_gen, 'max_q_mvar'] = 0.5/1000  # Example value, adjust as needed
    pv_generators.append(pv_gen)

In [11]:
# Define cost functions for PV generators to allow curtailment
for pv_gen in pv_generators:
    pp.create_poly_cost(net, pv_gen, 'sgen', cp1_eur_per_mw=-1000)  # Linear cost

# Initialize ConstControl for PV and load profiles with correct time steps
const_pv = ConstControl(net, element='sgen', element_index=pv_generators,
                        variable='p_mw', data_source=ds_pv, profile_name=["pvgen"] * len(pv_generators))

In [12]:
# Define the OPF function
def run_opf(net):
    try:
        pp.runopp(net, verbose=True)
        cost = net.res_cost  # Extract the cost from the OPF results
        return True, True, cost  # OPF and control converged
    except pp.optimal_powerflow.OPFNotConverged:
        print("OPF did not converge. Please check the network configuration and constraints.")
        return False, False, None  # OPF and control did not converge

In [13]:
# Create data storage for manual logging
results = {
    "time_step": [],
    "bus_vm_pu": [],
    "line_loading_percent": [],
    "load_p_mw": [],
    "load_q_mvar": [],
    "sgen_p_mw": [],
    "sgen_q_mvar": [],
    "cost": []
}

In [14]:
# Manually run the time series simulation
time_steps = df_pv.index  # Use the actual time step index from df_pv
for t in time_steps:
    current_time = df_pv.loc[t, 'time'].strftime('%H:%M:%S')  # Convert the time to a string
    
    # Update the PV generation limits for each time step
    for i, pv_gen in enumerate(pv_generators):
        max_p = df_pv.loc[t, 'pvgen']
        net.sgen.at[pv_gen, 'max_p_mw'] = max_p
    
    # Update the control objects to the current time step
    const_pv.time_step(net, time=t)  # Use the time step index `t`
    const_load.time_step(net, time=t)  # Use the time step index `t`
    
    # Run OPF and get convergence status
    pf_converged, ctrl_converged, cost = run_opf(net)

    # Log results for the current time step
    results["time_step"].append(t)
    results["bus_vm_pu"].append(net.res_bus.vm_pu.values)
    results["line_loading_percent"].append(net.res_line.loading_percent.values)
    results["load_p_mw"].append(net.res_load.p_mw.values)
    results["load_q_mvar"].append(net.res_load.q_mvar.values)
    results["sgen_p_mw"].append(net.res_sgen.p_mw.values)
    results["sgen_q_mvar"].append(net.res_sgen.q_mvar.values)
    results["cost"].append(cost)
    

# Convert results to DataFrame and save to Excel
results_df = pd.DataFrame({
    "time_step": results["time_step"],
    "bus_vm_pu": [list(vm_pu) for vm_pu in results["bus_vm_pu"]],
    "line_loading_percent": [list(loading_percent) for loading_percent in results["line_loading_percent"]],
    "load_p_mw": [list(load_p_mw) for load_p_mw in results["load_p_mw"]],
    "load_q_mvar": [list(load_q_mvar) for load_q_mvar in results["load_q_mvar"]],
    "sgen_p_mw": [list(sgen_p_mw) for sgen_p_mw in results["sgen_p_mw"]],
    "sgen_q_mvar": [list(sgen_q_mvar) for sgen_q_mvar in results["sgen_q_mvar"]],
    "cost": results["cost"]
})
results_df.to_excel("output_results.xlsx", index=False)

PYPOWER Version 5.1.4, 27-June-2018 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.59 seconds
Objective Function Value = 0.00 $/hr
| PyPower (ppci) System Summary - these are not valid for pandapower DataFrames|

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             19     Total Gen Capacity   1000000000.0       -1000000000.0 to 1000000000.0
Generators         6     On-line Capacity     1000000000.0       -1000000000.0 to 1000000000.0
Committed Gens     6     Generation (actual)      0.0               0.1
Loads              6     Load                     0.0               0.1
  Fixed            6       Fixed                  0.0               0.1
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches        

KeyboardInterrupt: 

In [ ]:
import ast

# Check and convert string representations of lists back to actual lists if necessary
def convert_to_list_if_needed(cell):
    if isinstance(cell, str):
        return eval(cell)
    return cell

results_df['bus_vm_pu'] = results_df['bus_vm_pu'].apply(convert_to_list_if_needed)
results_df['line_loading_percent'] = results_df['line_loading_percent'].apply(convert_to_list_if_needed)
results_df['load_p_mw'] = results_df['load_p_mw'].apply(convert_to_list_if_needed)
results_df['load_q_mvar'] = results_df['load_q_mvar'].apply(convert_to_list_if_needed)
results_df['sgen_p_mw'] = results_df['sgen_p_mw'].apply(convert_to_list_if_needed)
results_df['sgen_q_mvar'] = results_df['sgen_q_mvar'].apply(convert_to_list_if_needed)
results_df['cost'] = results_df['cost'].apply(convert_to_list_if_needed)

In [ ]:
# Create individual DataFrames for each result type
vm_pu = pd.DataFrame(results_df['bus_vm_pu'].tolist(), index=results_df['time_step'])
loading_percent = pd.DataFrame(results_df['line_loading_percent'].tolist(), index=results_df['time_step'])
load_p_mw = pd.DataFrame(results_df['load_p_mw'].tolist(), index=results_df['time_step'])
load_q_mvar = pd.DataFrame(results_df['load_q_mvar'].tolist(), index=results_df['time_step'])
sgen_p_mw = pd.DataFrame(results_df['sgen_p_mw'].tolist(), index=results_df['time_step'])
sgen_q_mvar = pd.DataFrame(results_df['sgen_q_mvar'].tolist(), index=results_df['time_step'])
cost = pd.DataFrame(results_df['cost'].tolist(), index=results_df['time_step'])


In [ ]:
import matplotlib.pyplot as plt
# Generate the plots
plt.figure(figsize=(10, 5))
for column in vm_pu.columns:
    plt.plot(vm_pu.index, vm_pu[column], label=f'Bus {column}')
plt.xlabel('Time Step')
plt.ylabel('Voltage Magnitude (p.u.)')
plt.title('Voltage Magnitude over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
for column in loading_percent.columns:
    plt.plot(loading_percent.index, loading_percent[column], label=f'Line {column}')
plt.xlabel('Time Step')
plt.ylabel('Line Loading (%)')
plt.title('Line Loading over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
for column in load_p_mw.columns:
    plt.plot(load_p_mw.index, load_p_mw[column], label=f'Load {column}')
plt.xlabel('Time Step')
plt.ylabel('Load Power (MW)')
plt.title('Load Power over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
for column in sgen_p_mw.columns:
    plt.plot(sgen_p_mw.index, sgen_p_mw[column], label=f'SGen {column}')
plt.xlabel('Time Step')
plt.ylabel('SGen Power (MW)')
plt.title('SGen Power over Time')
plt.legend()
plt.show()

In [ ]:
# Plot the cost over time
plt.figure(figsize=(10, 5))
plt.plot(results_df['time_step'], results_df['cost']*-1, label='Cost')
plt.xlabel('Time Step')
plt.ylabel('Cost ($/hr)')
plt.title('Cost over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(sgen_p_mw, label=sgen_p_mw.columns)
plt.xlabel('Time Step')
plt.ylabel('Active Power Load (MW)')
plt.title('Active PV Power over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(load_p_mw, label=load_p_mw.columns)
plt.xlabel('Time Step')
plt.ylabel('Active Power Load (MW)')
plt.title('Active Power Load over Time')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Print the first few rows of the PV generation profile
print(df_pv.head())

# Plot the PV generation profile
plt.plot(df_pv['pvgen'])
plt.title('PV Generation Profile')
plt.xlabel('Time Step')
plt.ylabel('MW')
plt.show()